<a href="https://colab.research.google.com/github/tatsath/Interpretability/blob/main/sentiment_steering.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [55]:
import goodfire

GOODFIRE_API_KEY = "sk-goodfire-dmKRWrIVThY26TCbxycGJjAO3q2gvxL2S3LBXuQk7ROn_qiJsk2CNA"
client = goodfire.Client(api_key=GOODFIRE_API_KEY)

In [56]:
variant = goodfire.Variant("meta-llama/Meta-Llama-3.1-8B-Instruct")

In [57]:
# import pandas as pd

# df = pd.read_excel("sentence_sentiment_gpt4o.xlsx")
# df.head()

In [58]:
df = pd.read_excel('https://raw.githubusercontent.com/ashater/sae/master/sentence_sentiment_gpt4o.xlsx')

In [72]:
df.head()

,Version,Category,Sentiment,Sentence
0,v1,Revenue Growth,Very Positive,The company reported record-breaking revenue g...
1,v1,Revenue Growth,Very Positive,"Due to its innovative approach, the firm has s..."
2,v1,Debt Management,Very Positive,The early repayment of $2 billion in outstandi...
3,v1,Debt Management,Very Positive,The firm’s bonds have been upgraded to AAA sta...
4,v1,Market Performance,Very Positive,Investors are thrilled as the company’s stock ...


In [80]:
# prompt: get distinct columns from the Sentiment column of this df

import pandas as pd

df = pd.read_excel('https://raw.githubusercontent.com/ashater/sae/master/sentence_sentiment_gpt4o.xlsx')
distinct_sentiments = df['Sentiment'].unique()
distinct_sentiments


array(['Very Positive', 'Very Negative', 'Neutral', 'Somewhat Positive',
       'Somewhat Negative'], dtype=object)

In [59]:
df["Sentence"][0]

'The company reported record-breaking revenue growth of 40% this quarter, driven by robust sales and successful product launches.'

In [94]:
system_prompt = """\
You are an intelligent AI Assistant and your task is to provide a sentiment for the sentence provided.\
Reply with the sentiment only in one out of these five categories - 'Very Positive', 'Very Negative', 'Neutral', 'Somewhat Positive',
       'Somewhat Negative' . No explanation or "." is required.
"""

In [95]:
def get_model_response(prompt, model_variant, system_prompt=None):
    messages = [{"role": "user", "content": prompt}]
    try:
        response = client.chat.completions.create(
            messages=messages,
            model=model_variant,
            max_completion_tokens=50,
            system_prompt=system_prompt
        )
        return response.choices[0].message["content"].strip()
    except Exception as e:
        print(f"Error during completion: {e}")
        return "ERROR"


get_model_response(df["Sentence"][999], variant, system_prompt)

'Very Positive'

In [96]:
# Option 2 - steer on sentence category
query = "credit risk related"

features = client.features.search(query=query, model=variant, top_k=10)
features

FeatureGroup([
   0: "Financial credit and banking terminology",
   1: "Credit ratings and financial strength assessments",
   2: "Risk assessment in insurance underwriting",
   3: "Discussions of credit scores and credit history, especially concerns and improvement",
   4: "Business and financial risk assessment terminology",
   5: "Identity theft and financial fraud risks",
   6: "Trading risk warnings and responsible money management advice",
   7: "Bankruptcy and insolvency proceedings across languages",
   8: "The semantic root 'cred' relating to trust, authentication, and financial credit across languages",
   9: "Probabilistic financial loss estimation in risk assessment forms"
])

In [98]:
get_model_response(df["Sentence"][1], variant, system_prompt)

'Very Positive'

In [86]:
# results = []
# for i in range(1):
#     response = get_model_response(df["Sentence"][i], variant, system_prompt)
#     results.append(response)

# # Print the results
# print(results)

In [99]:
import pandas as pd

# Assuming you have already loaded your DataFrame as 'df' and have the 'results' list from the previous code

# Create a new DataFrame with three columns
compared_df = pd.DataFrame({
    'sen_index': df.index[:len(results)],  # Index of the sentence
    'sentiment_annotated': df['Sentiment'][:len(results)],  # Sentiment from DataFrame
    'sentence_steered': results  # Sentiment from model output
})

# Display the new DataFrame
print(compared_df)

Empty DataFrame
Columns: [sen_index, sentiment_annotated, sentence_steered]
Index: []


In [100]:
import pandas as pd
import random

# Assuming you have already loaded your DataFrame as 'df' and have the necessary functions

compared_df = pd.DataFrame(columns=['index', 'sentiment_annotated', 'sentence_steered'])  # Create an empty DataFrame

num_sentences_to_select = 10  # Number of random sentences to select

for _ in range(num_sentences_to_select):
    random_index = random.choice(df.index)  # Select a random index
    sentence = df["Sentence"][random_index]  # Get the sentence at the random index
    response = get_model_response(sentence, variant, system_prompt)  # Compute sentiment

    # Append the results to the DataFrame
    new_row = pd.DataFrame({
        'index': [random_index],
        'sentiment_annotated': [df['Sentiment'][random_index]],
        'sentence_steered': [response]
    })
    compared_df = pd.concat([compared_df, new_row], ignore_index=True)

# Display the DataFrame
print(compared_df)

  index sentiment_annotated   sentence_steered
0  4536             Neutral  Somewhat Positive
1  3426       Very Negative      Very Negative
2  2000   Somewhat Positive  Somewhat Positive
3  4137       Very Positive      Very Positive
4  5080   Somewhat Positive      Very Positive
5   352       Very Positive      Very Positive
6  3545             Neutral      Very Positive
7  2401   Somewhat Positive  Somewhat Positive
8  2089   Somewhat Positive            Neutral
9  2205   Somewhat Positive      Very Positive


In [106]:
import pandas as pd
import random

# Assuming you have already loaded your DataFrame as 'df' and have the necessary functions

compared_df = pd.DataFrame(columns=['index', 'sentiment_annotated', 'sentence_not_steered', 'sentence_steered' ])

num_sentences_to_select = 5

for _ in range(num_sentences_to_select):
    random_index = random.choice(df.index)
    sentence = df["Sentence"][random_index]

    # Get steered response
    response_not_steered = get_model_response(sentence, variant, system_prompt)

    # Reset steering and get non-steered response
    variant.reset()  # Reset steering for non-steered output
    # variant.reset()
    variant.set(features[1], 1)
    response_steered = get_model_response(sentence, variant, system_prompt)

    # Append results to DataFrame
    new_row = pd.DataFrame({
        'index': [random_index],
        'sentiment_annotated': [df['Sentiment'][random_index]],
        'sentence_not_steered': [response_not_steered],
        'sentence_steered': [response_steered],
    })
    compared_df = pd.concat([compared_df, new_row], ignore_index=True)

# Display the DataFrame
print(compared_df)

  index sentiment_annotated  \
0  5281             Neutral   
1  1598   Somewhat Positive   
2  1647   Somewhat Negative   
3  5576   Somewhat Positive   
4  3556             Neutral   

                                sentence_not_steered    sentence_steered  
0  ratingumbing ratings rating rating ratings rat...      Very Positive.  
1                                     Very Positive.      Very Positive.  
2                                  Somewhat Negative   Somewhat Negative  
3                                     Very Positive.      Very Positive.  
4                                 Somewhat Positive.  Somewhat Positive.  
